# [Module 3.1] Personalize 솔류션 생성

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.

* 레서피(알고리즘) 선택 및 솔류션 생성
* 솔류션 버전 생성
* 솔류션 평가 지표 얻기



In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
# suffix = str(np.random.uniform())[4:9]

## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

이 LAB에서 아래와 같은 솔루션을 생성하여 성능을 비교하여 봅니다.

    1) HRNN
    2) HRNN Coldstart
    3) SIMS
    4) Personalized Ranking

이 프로세스의 완료는 실제로 40분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

###  레시피 리스트 확인

In [5]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


###  User-Personalization 솔루션 생성 

In [6]:
# Recipe 선택 
system_user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-user-personalization-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_user_personalization_recipe_arn,
)

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-91891",
  "ResponseMetadata": {
    "RequestId": "734f6ca6-04ba-4207-9bc9-717f109c172a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:24 GMT",
      "x-amzn-requestid": "734f6ca6-04ba-4207-9bc9-717f109c172a",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [25]:
user_personalization_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-91891",
  "ResponseMetadata": {
    "RequestId": "1c41a038-5351-489c-94f6-3b7cf2841d36",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 08:49:14 GMT",
      "x-amzn-requestid": "1c41a038-5351-489c-94f6-3b7cf2841d36",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [7]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_recipe_arn
)

user_personalization_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-91891/5ad2fc30",
  "ResponseMetadata": {
    "RequestId": "bb407da1-164b-48d2-9a8e-456043c31b45",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:24 GMT",
      "x-amzn-requestid": "bb407da1-164b-48d2-9a8e-456043c31b45",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


###  Popularity Count 솔루션 생성 

In [8]:
# Recipe 선택 
system_popularity_recipe_arn = "arn:aws:personalize:::recipe/aws-popularity-count"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-popularity-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_popularity_recipe_arn,
)



{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-91891",
  "ResponseMetadata": {
    "RequestId": "f536b8e4-ce3e-49c2-9bd2-b64e9c5ce435",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:26 GMT",
      "x-amzn-requestid": "f536b8e4-ce3e-49c2-9bd2-b64e9c5ce435",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [26]:
popularity_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-91891",
  "ResponseMetadata": {
    "RequestId": "1c41a038-5351-489c-94f6-3b7cf2841d36",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 08:49:14 GMT",
      "x-amzn-requestid": "1c41a038-5351-489c-94f6-3b7cf2841d36",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [9]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = popularity_recipe_arn
)

popularity_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-91891/51d69553",
  "ResponseMetadata": {
    "RequestId": "163ea735-c7f3-4da3-9531-42de17502da3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:27 GMT",
      "x-amzn-requestid": "163ea735-c7f3-4da3-9531-42de17502da3",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN 솔루션 생성

In [11]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

hrnn_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-91891/47b38681",
  "ResponseMetadata": {
    "RequestId": "cf3f664b-417a-43c6-9271-54f8aa2910cd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:28 GMT",
      "x-amzn-requestid": "cf3f664b-417a-43c6-9271-54f8aa2910cd",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN-Meta 솔루션 생성

In [22]:
 # Recipe 선택 
system_hrnn_meta_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-metadata"


# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-aws-hrnn-metadata-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_meta_recipe_arn,
)

hrnn_meta_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-91891",
  "ResponseMetadata": {
    "RequestId": "1c41a038-5351-489c-94f6-3b7cf2841d36",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 08:49:14 GMT",
      "x-amzn-requestid": "1c41a038-5351-489c-94f6-3b7cf2841d36",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [23]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_meta_solution_arn
)

hrnn_meta_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-91891/adaee5f0",
  "ResponseMetadata": {
    "RequestId": "0e855859-7fb4-4df2-b7cf-4ec6c4471a1a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 08:49:19 GMT",
      "x-amzn-requestid": "0e855859-7fb4-4df2-b7cf-4ec6c4471a1a",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN ColdStart 솔루션 생성

In [12]:
 # Recipe 선택 
system_hrnn_coldstart_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-coldstart"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-hrnn-coldstart-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_coldstart_recipe_arn,
    solutionConfig = {
        "featureTransformationParameters" : {
            'cold_start_max_duration' : '5',
            'cold_start_relative_from' : 'latestItem',
            'cold_start_max_interactions':'15'
        }
    }
)

hrnn_coldstart_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-coldstart-91891",
  "ResponseMetadata": {
    "RequestId": "b15768e2-deda-413d-83ae-e099708c6ad7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:30 GMT",
      "x-amzn-requestid": "b15768e2-deda-413d-83ae-e099708c6ad7",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [13]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_coldstart_solution_arn
)

hrnn_coldstart_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-coldstart-91891/5afbfa75",
  "ResponseMetadata": {
    "RequestId": "81fbe2b8-31f4-49a9-988d-7945be4dbc12",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:31 GMT",
      "x-amzn-requestid": "81fbe2b8-31f4-49a9-988d-7945be4dbc12",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS 솔루션 생성 

In [14]:
 # Recipe 선택 
system_sims_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-sims-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_sims_recipe_arn
)

sims_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-91891",
  "ResponseMetadata": {
    "RequestId": "d766fd94-891e-4210-96f9-cedd28af75e5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:32 GMT",
      "x-amzn-requestid": "d766fd94-891e-4210-96f9-cedd28af75e5",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [15]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)
sims_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-91891/84632efc",
  "ResponseMetadata": {
    "RequestId": "6b1df847-dd34-46ce-bf3c-33279e30cbf6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:32 GMT",
      "x-amzn-requestid": "6b1df847-dd34-46ce-bf3c-33279e30cbf6",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalize Ranking 솔루션 생성 

In [16]:
 # Recipe 선택 
system_ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-ranking-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_ranking_recipe_arn
)

ranking_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-91891",
  "ResponseMetadata": {
    "RequestId": "0440919d-7aea-44d1-99a2-efa2ee3c91c8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:35 GMT",
      "x-amzn-requestid": "0440919d-7aea-44d1-99a2-efa2ee3c91c8",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [17]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = ranking_solution_arn
)
ranking_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-91891/04a98ddc",
  "ResponseMetadata": {
    "RequestId": "b1a14a3c-393e-4316-8aec-a93711c714a0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:57:34 GMT",
      "x-amzn-requestid": "b1a14a3c-393e-4316-8aec-a93711c714a0",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 모델 생성 확인 하기 
아래 코드 실행하여 모델 생성 여부를 확인합니다. 

In [18]:
%%time


max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    ##user_personalization Status
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = user_personalization_solution_version_arn
    )  
    status_user_per = describe_solution_version_response["solutionVersion"]["status"]
    print("user_personalization SolutionVersion: {}".format(status_user_per))
    
    ##Popularity Status
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = popularity_solution_version_arn
    )  
    status_pop = describe_solution_version_response["solutionVersion"]["status"]
    print("Popularity SolutionVersion: {}".format(status_pop))
    
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_solution_version_arn
    )  
    status_hrnn = describe_solution_version_response["solutionVersion"]["status"]
    print("HRNN SolutionVersion: {}".format(status_hrnn))

    #hrnn_meta status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_meta_solution_version_arn
    )
    status_hrnn_meta = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN Meta SolutionVersion: {}".format(status_hrnn_meta))
    
    
    #hrnn_coldstart status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_coldstart_solution_version_arn
    )
    status_hrnn_cs = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN ColdStart SolutionVersion: {}".format(status_hrnn_cs))
    
    #sims status
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = sims_solution_version_arn
    )
    status_sims = describe_solution_version_response["solutionVersion"]["status"]
    print("Sims SolutionVersion: {}".format(status_sims))
    
    #ranking status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = ranking_solution_version_arn
    )
    status_ranking= describe_solution_version_response["solutionVersion"]["status"]    
    print("Ranking SolutionVersion: {}".format(status_ranking))    
    
    if (status_user_per == "ACTIVE" or status_user_per == "CREATE FAILED") &\
       (status_pop == "ACTIVE" or status_pop == "CREATE FAILED") &\
       (status_hrnn == "ACTIVE" or status_hrnn == "CREATE FAILED") &\
       (status_hrnn_meta == "ACTIVE" or status_hrnn_meta == "CREATE FAILED") &\    
       (status_hrnn_cs == "ACTIVE" or status_hrnn_cs == "CREATE FAILED") &\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED") &\
       (status_ranking == "ACTIVE" or status_ranking == "CREATE FAILED"):
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All solution creation completed")

user_personalization SolutionVersion: CREATE PENDING
Popularity SolutionVersion: CREATE PENDING
HRNN SolutionVersion: CREATE PENDING
HRNN ColdStart SolutionVersion: CREATE PENDING
Sims SolutionVersion: CREATE PENDING
Ranking SolutionVersion: CREATE PENDING
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popu

### Store variables

In [29]:

%store user_personalization_solution_arn
%store user_personalization_solution_version_arn

%store popularity_solution_arn
%store popularity_solution_version_arn

%store hrnn_solution_version_arn
%store hrnn_solution_arn

%store hrnn_meta_solution_version_arn
%store hrnn_meta_solution_arn


%store hrnn_coldstart_solution_version_arn
%store hrnn_coldstart_solution_arn

%store sims_solution_version_arn
%store sims_solution_arn

%store ranking_solution_version_arn
%store ranking_solution_arn


Stored 'user_personalization_solution_arn' (str)
Stored 'user_personalization_solution_version_arn' (str)
Stored 'popularity_solution_arn' (str)
Stored 'popularity_solution_version_arn' (str)
Stored 'hrnn_solution_version_arn' (str)
Stored 'hrnn_solution_arn' (str)
Stored 'hrnn_meta_solution_version_arn' (str)
Stored 'hrnn_meta_solution_arn' (str)
Stored 'hrnn_coldstart_solution_version_arn' (str)
Stored 'hrnn_coldstart_solution_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'ranking_solution_version_arn' (str)
Stored 'ranking_solution_arn' (str)
